In [ ]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

In [ ]:
stats_Request = requests.get("https://www.baseball-reference.com/leagues/majors/2021-starter-pitching.shtml")

In [ ]:
stats_soup = BeautifulSoup(stats_Request.text)
pitching_table = stats_soup.select('#teams_starter_pitching .left, #teams_starter_pitching .center, #teams_starter_pitching .right')

In [ ]:
colnames = ["Year"]
for colname in pitching_table[0:33]:
  colnames.append(colname.text)

starting_pitching = pd.DataFrame(columns=colnames)
starting_pitching

,Year,Tm,Arizona Diamondbacks,Atlanta Braves,Baltimore Orioles,Boston Red Sox,Chicago Cubs,Chicago White Sox,Cincinnati Reds,Cleveland Indians,...,San Diego Padres,Seattle Mariners,San Francisco Giants,St. Louis Cardinals,Tampa Bay Rays,Texas Rangers,Toronto Blue Jays,Washington Nationals,League Average,


In [ ]:
for year in range(2000, 2022):
   stats_Request = requests.get(f"https://www.baseball-reference.com/leagues/majors/{year}-starter-pitching.shtml")
   stats_soup = BeautifulSoup(stats_Request.text)
   pitching_table = stats_soup.select('#teams_starter_pitching .center , #teams_starter_pitching .left, #teams_starter_pitching .right')
   starting_pos = 0
   ending_pos = 33
   for row in range(30):
     starting_pos += 33
     ending_pos += 33
     new_row = [year]
     for element in pitching_table[starting_pos:ending_pos]:
       new_row.append(element.text)
     row_series = pd.Series(new_row, index = colnames)
     starting_pitching = starting_pitching.append(row_series, ignore_index = True)
starting_pitching

,Year,Tm,Arizona Diamondbacks,Atlanta Braves,Baltimore Orioles,Boston Red Sox,Chicago Cubs,Chicago White Sox,Cincinnati Reds,Cleveland Indians,...,San Diego Padres,Seattle Mariners,San Francisco Giants,St. Louis Cardinals,Tampa Bay Rays,Texas Rangers,Toronto Blue Jays,Washington Nationals,League Average,
0,2000,GS,Wgs,Lgs,ND,Wchp,Ltuf,Wtm,Ltm,tmW-L%,...,RS/IP,IP/GS,Pit/GS,<80,80-99,100-119,≥120,Max,Tm,Anaheim Angels
1,2000,Arizona Diamondbacks,Atlanta Braves,Baltimore Orioles,Boston Red Sox,Chicago Cubs,Chicago White Sox,Cincinnati Reds,Cleveland Indians,Colorado Rockies,...,Seattle Mariners,San Francisco Giants,St. Louis Cardinals,Tampa Bay Devil Rays,Texas Rangers,Toronto Blue Jays,League Average,,Tm,162
2,2000,162,53,56,53,16,12,82,80,.506,...,5.2,5.5,92,35,56,64,7,132,162,162
3,2000,62,53,47,14,13,85,77,.525,16,...,6.2,95,28,67,45,22,145,162,162,76
4,2000,49,37,18,12,95,67,.586,14,18,...,96,23,71,61,7,140,162,162,52,67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
655,2021,3.8,5.1,85,43,91,28,0,113,162,...,39%,55,90,0,188,63,9,86,5.1,4.6
656,2021,5.1,81,52,92,18,0,118,162,162,...,51,90,-3,272,102,1,103,4.5,4.1,5.1
657,2021,84,45,94,23,0,113,162,162,42,...,88,-5,163,46,12,109,5.4,4.3,4.6,73
658,2021,88,55,19,0,114,162,162,33,71,...,-3,221,73,4,102,3.9,3.4,4.9,81,69


In [ ]:
playoff_Request = requests.get('https://www.baseball-reference.com/postseason/')
playoff_soup = BeautifulSoup(playoff_Request.text)
playoff_data = playoff_soup.select('#postseason_series a')

In [ ]:
starting_pos = 0 
ending_pos = 3
post_names = ['Year', 'Winner', 'Loser']
post_teams = pd.DataFrame(columns=post_names)

for playoff_series in range(1, 181):
    post_list = []
    for element in playoff_data[starting_pos:ending_pos]:
        post_list.append(element.text)
    post_Series = pd.Series(post_list, index=post_names)
    post_teams = post_teams.append(post_Series, ignore_index=True)
    starting_pos += 3
    ending_pos += 3

post_teams.Year = post_teams.Year.str.extract('(\d{4})+')
post_teams.Year = post_teams.Year.astype('int')
post_teams.Winner = post_teams.Winner.str.extract('([\w\s]+\.?[\w\s]+)\*? \(')
post_teams.Loser = post_teams.Loser.str.extract('([\w\s]+\.?[\w\s]+)\*? \(')
post_teams

,Year,Winner,Loser
0,2021,Atlanta Braves,Houston Astros
1,2021,Houston Astros,Boston Red Sox
2,2021,Atlanta Braves,Los Angeles Dodgers
3,2021,Houston Astros,Chicago White Sox
4,2021,Boston Red Sox,Tampa Bay Rays
...,...,...,...
175,2000,New York Mets,St. Louis Cardinals
176,2000,New York Yankees,Oakland Athletics
177,2000,Seattle Mariners,Chicago White Sox
178,2000,St. Louis Cardinals,Atlanta Braves


In [ ]:
playoff_status = []
for year in range(2000, 2022):
    year_teams = starting_pitching.loc[starting_pitching['Year'] == year, 'Tm'].unique()
    year_post_teams = post_teams.loc[post_teams['Year'] == year, :]
    unique_year_post_teams = pd.concat([year_post_teams.Winner, year_post_teams.Loser], ignore_index=True).unique()
    for team in year_teams:
        if team in unique_year_post_teams:
            playoff_status.append(1)
        else:
            playoff_status.append(0)
            
starting_pitching['Playoff'] = pd.Series(playoff_status)

In [ ]:
starting_pitching.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 660 entries, 0 to 659
Data columns (total 35 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Year     660 non-null    object
 1   Tm       660 non-null    object
 2   G        660 non-null    object
 3   GS       660 non-null    object
 4   Wgs      660 non-null    object
 5   Lgs      660 non-null    object
 6   ND       660 non-null    object
 7   Wchp     660 non-null    object
 8   Ltuf     660 non-null    object
 9   Wtm      660 non-null    object
 10  Ltm      660 non-null    object
 11  tmW-L%   660 non-null    object
 12  Wlst     660 non-null    object
 13  Lsv      660 non-null    object
 14  CG       660 non-null    object
 15  SHO      660 non-null    object
 16  QS       660 non-null    object
 17  QS%      660 non-null    object
 18  GmScA    660 non-null    object
 19  Best     660 non-null    object
 20  Wrst     660 non-null    object
 21  BQR      660 non-null    object
 22  BQ

In [ ]:
to_numeric_columns = ['G','GS','Wgs','Lgs','ND','Wchp','Ltuf','Wtm','Ltm','Wlst','Lsv','CG','SHO','QS','GmScA','Best','Wrst','BQR','BQS','sDR','lDR','Pit/GS','<80','80-99','100-119','≥120','Max', 'tmW-L%','RS/GS','RS/IP','IP/GS']

In [ ]:
starting_pitching[to_numeric_columns] = starting_pitching[to_numeric_columns].apply(pd.to_numeric, errors='coerce', axis=1)
starting_pitching

,Year,Tm,G,GS,Wgs,Lgs,ND,Wchp,Ltuf,Wtm,...,RS/GS,RS/IP,IP/GS,Pit/GS,<80,80-99,100-119,≥120,Max,Playoff
0,2000,Anaheim Angels,162.0,162.0,53.0,56.0,53.0,16.0,12.0,82.0,...,5.4,5.2,5.5,92.0,35.0,56.0,64.0,7.0,132.0,0
1,2000,Arizona Diamondbacks,162.0,162.0,62.0,53.0,47.0,14.0,13.0,85.0,...,4.9,4.8,6.2,95.0,28.0,67.0,45.0,22.0,145.0,0
2,2000,Atlanta Braves,162.0,162.0,76.0,49.0,37.0,18.0,12.0,95.0,...,5.1,4.8,6.4,96.0,23.0,71.0,61.0,7.0,140.0,1
3,2000,Baltimore Orioles,162.0,162.0,52.0,67.0,43.0,14.0,17.0,74.0,...,5.0,4.9,6.2,102.0,19.0,45.0,67.0,31.0,137.0,0
4,2000,Boston Red Sox,162.0,162.0,56.0,54.0,52.0,17.0,14.0,85.0,...,4.9,4.7,5.5,89.0,48.0,67.0,38.0,9.0,134.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
655,2021,St. Louis Cardinals,162.0,162.0,54.0,47.0,61.0,19.0,5.0,90.0,...,4.5,4.1,5.1,84.0,45.0,94.0,23.0,0.0,113.0,1
656,2021,Tampa Bay Rays,162.0,162.0,42.0,30.0,90.0,24.0,4.0,100.0,...,5.4,4.3,4.6,73.0,88.0,55.0,19.0,0.0,114.0,1
657,2021,Texas Rangers,162.0,162.0,33.0,71.0,58.0,14.0,10.0,60.0,...,3.9,3.4,4.9,81.0,69.0,76.0,16.0,1.0,124.0,0
658,2021,Toronto Blue Jays,162.0,162.0,61.0,40.0,61.0,20.0,6.0,91.0,...,5.5,5.6,5.2,85.0,46.0,82.0,34.0,0.0,114.0,0


In [ ]:
starting_pitching['QS%'] = starting_pitching['QS%'].map(lambda x: x.rstrip('%')).astype('int')
starting_pitching['Year'] = starting_pitching['Year'].astype('int')

,Year,Tm,G,GS,Wgs,Lgs,ND,Wchp,Ltuf,Wtm,...,RS/GS,RS/IP,IP/GS,Pit/GS,<80,80-99,100-119,≥120,Max,Playoff
0,2000,Anaheim Angels,162.0,162.0,53.0,56.0,53.0,16.0,12.0,82.0,...,5.4,5.2,5.5,92.0,35.0,56.0,64.0,7.0,132.0,0
1,2000,Arizona Diamondbacks,162.0,162.0,62.0,53.0,47.0,14.0,13.0,85.0,...,4.9,4.8,6.2,95.0,28.0,67.0,45.0,22.0,145.0,0
2,2000,Atlanta Braves,162.0,162.0,76.0,49.0,37.0,18.0,12.0,95.0,...,5.1,4.8,6.4,96.0,23.0,71.0,61.0,7.0,140.0,1
3,2000,Baltimore Orioles,162.0,162.0,52.0,67.0,43.0,14.0,17.0,74.0,...,5.0,4.9,6.2,102.0,19.0,45.0,67.0,31.0,137.0,0
4,2000,Boston Red Sox,162.0,162.0,56.0,54.0,52.0,17.0,14.0,85.0,...,4.9,4.7,5.5,89.0,48.0,67.0,38.0,9.0,134.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
655,2021,St. Louis Cardinals,162.0,162.0,54.0,47.0,61.0,19.0,5.0,90.0,...,4.5,4.1,5.1,84.0,45.0,94.0,23.0,0.0,113.0,1
656,2021,Tampa Bay Rays,162.0,162.0,42.0,30.0,90.0,24.0,4.0,100.0,...,5.4,4.3,4.6,73.0,88.0,55.0,19.0,0.0,114.0,1
657,2021,Texas Rangers,162.0,162.0,33.0,71.0,58.0,14.0,10.0,60.0,...,3.9,3.4,4.9,81.0,69.0,76.0,16.0,1.0,124.0,0
658,2021,Toronto Blue Jays,162.0,162.0,61.0,40.0,61.0,20.0,6.0,91.0,...,5.5,5.6,5.2,85.0,46.0,82.0,34.0,0.0,114.0,0


In [ ]:
starting_pitching

,Year,Tm,G,GS,Wgs,Lgs,ND,Wchp,Ltuf,Wtm,...,RS/GS,RS/IP,IP/GS,Pit/GS,<80,80-99,100-119,≥120,Max,Playoff
0,2000,Anaheim Angels,162.0,162.0,53.0,56.0,53.0,16.0,12.0,82.0,...,5.4,5.2,5.5,92.0,35.0,56.0,64.0,7.0,132.0,0
1,2000,Arizona Diamondbacks,162.0,162.0,62.0,53.0,47.0,14.0,13.0,85.0,...,4.9,4.8,6.2,95.0,28.0,67.0,45.0,22.0,145.0,0
2,2000,Atlanta Braves,162.0,162.0,76.0,49.0,37.0,18.0,12.0,95.0,...,5.1,4.8,6.4,96.0,23.0,71.0,61.0,7.0,140.0,1
3,2000,Baltimore Orioles,162.0,162.0,52.0,67.0,43.0,14.0,17.0,74.0,...,5.0,4.9,6.2,102.0,19.0,45.0,67.0,31.0,137.0,0
4,2000,Boston Red Sox,162.0,162.0,56.0,54.0,52.0,17.0,14.0,85.0,...,4.9,4.7,5.5,89.0,48.0,67.0,38.0,9.0,134.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
655,2021,St. Louis Cardinals,162.0,162.0,54.0,47.0,61.0,19.0,5.0,90.0,...,4.5,4.1,5.1,84.0,45.0,94.0,23.0,0.0,113.0,1
656,2021,Tampa Bay Rays,162.0,162.0,42.0,30.0,90.0,24.0,4.0,100.0,...,5.4,4.3,4.6,73.0,88.0,55.0,19.0,0.0,114.0,1
657,2021,Texas Rangers,162.0,162.0,33.0,71.0,58.0,14.0,10.0,60.0,...,3.9,3.4,4.9,81.0,69.0,76.0,16.0,1.0,124.0,0
658,2021,Toronto Blue Jays,162.0,162.0,61.0,40.0,61.0,20.0,6.0,91.0,...,5.5,5.6,5.2,85.0,46.0,82.0,34.0,0.0,114.0,0
